In [63]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from gensim.models import Word2Vec 
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import word_tokenize
import warnings;
warnings.filterwarnings('ignore')
from collections import Counter 
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zhiti\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
fz_df = pd.read_csv('tnveddata_20211126.csv', sep=';',encoding='mbcs')
fz_df

,,KOD_TNVED_SPR,OPISANIE_SPR
0,1,101210000,ЛОШАДИ ЖИВЫЕ: ЧИСТОПОРОДНЫЕ ПЛЕМЕННЫЕ ЖИВОТНЫЕ
1,2,101291000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ, ПРОЧИЕ:"
2,3,101299000,ПРОЧИЕ ЛОШАДИ ЖИВЫЕ: ПРОЧИЕ
3,4,101300000,"ОСЛЫ, ЖИВЫЕ"
4,5,101900000,ПРОЧИЕ МУЛЫ И ЛОШАКИ ЖИВЫЕ
...,...,...,...
12318,12319,9703000000,ПОДЛИННИКИ СКУЛЬПТУР И СТАТУЭТОК ИЗ ЛЮБЫХ МАТЕ...
12319,12320,9704000000,"МАРКИ ПОЧТОВЫЕ ИЛИ МАРКИ ГОСПОШЛИН, ЗНАКИ ПОЧТ..."
12320,12321,9705000000,КОЛЛЕКЦИИ И ПРЕДМЕТЫ КОЛЛЕКЦИОНИРОВАНИЯ ПО ЗОО...
12321,12322,9706000000,АНТИКВАРИАТ ВОЗРАСТОМ БОЛЕЕ 100 ЛЕТ


In [68]:
fz_df['KOD_SHORT'] = fz_df['KOD_TNVED_SPR'].apply(lambda x: str(x)[:4])
fz_df

,,KOD_TNVED_SPR,OPISANIE_SPR,KOD_SHORT
0,1,101210000,ЛОШАДИ ЖИВЫЕ: ЧИСТОПОРОДНЫЕ ПЛЕМЕННЫЕ ЖИВОТНЫЕ,1012
1,2,101291000,"ЛОШАДИ, ОСЛЫ, МУЛЫ И ЛОШАКИ ЖИВЫЕ, ПРОЧИЕ:",1012
2,3,101299000,ПРОЧИЕ ЛОШАДИ ЖИВЫЕ: ПРОЧИЕ,1012
3,4,101300000,"ОСЛЫ, ЖИВЫЕ",1013
4,5,101900000,ПРОЧИЕ МУЛЫ И ЛОШАКИ ЖИВЫЕ,1019
...,...,...,...,...
12318,12319,9703000000,ПОДЛИННИКИ СКУЛЬПТУР И СТАТУЭТОК ИЗ ЛЮБЫХ МАТЕ...,9703
12319,12320,9704000000,"МАРКИ ПОЧТОВЫЕ ИЛИ МАРКИ ГОСПОШЛИН, ЗНАКИ ПОЧТ...",9704
12320,12321,9705000000,КОЛЛЕКЦИИ И ПРЕДМЕТЫ КОЛЛЕКЦИОНИРОВАНИЯ ПО ЗОО...,9705
12321,12322,9706000000,АНТИКВАРИАТ ВОЗРАСТОМ БОЛЕЕ 100 ЛЕТ,9706


In [65]:
user_df = pd.read_csv('dataset_20211126.csv', sep=';',encoding='mbcs')
user_df.drop_duplicates()
user_df

,TNVED,OPISANIE,DATA
0,9025,"ЧАСТИ И ПРИНАДЛЕЖНОСТИ АППАРАТУРЫ ПОЗ.9025, НЕ...",07.09.2021
1,8504,"ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ Б...",07.09.2021
2,8481,"КЛАПАНЫ ЗАПОРНЫЕ ИЗ СТАЛИ, НЕ ВОЕННОГО НАЗНАЧЕ...",07.09.2021
3,8531,УСТРОЙСТВА СИГНАЛИЗАЦИОННЫЕ ОХРАННЫЕ И УСТРОЙС...,07.09.2021
4,3402,"ВЕЩЕСТВА ПОВЕРХНОСТНО-АКТИВНЫЕ ОРГАНИЧЕСКИЕ, А...",02.09.2021
...,...,...,...
4177894,2710,"МИНЕРАЛЬНОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ Д...",24.10.2021
4177895,3403,"СИНТЕТИЧЕСКОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ...",24.10.2021
4177896,3403,"СИНТЕТИЧЕСКОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ...",24.10.2021
4177897,3403,"МАСЛО ТРАНСМИССИОННОЕ НА СИНТЕТИЧЕСКОЙ ОСНОВЕ,...",24.10.2021


In [73]:
user_df.drop(['DATA'], axis=1, inplace=True)
user_df

,TNVED,OPISANIE,clean_sentence
0,9025,"ЧАСТИ И ПРИНАДЛЕЖНОСТИ АППАРАТУРЫ ПОЗ.9025, НЕ...",части и принадлежности аппаратуры поз 9025 не...
1,8504,"ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ Б...",трансформаторы мощностью более 1 ква но не б...
2,8481,"КЛАПАНЫ ЗАПОРНЫЕ ИЗ СТАЛИ, НЕ ВОЕННОГО НАЗНАЧЕ...",клапаны запорные из стали не военного назначе...
3,8531,УСТРОЙСТВА СИГНАЛИЗАЦИОННЫЕ ОХРАННЫЕ И УСТРОЙС...,устройства сигнализационные охранные и устройс...
4,3402,"ВЕЩЕСТВА ПОВЕРХНОСТНО-АКТИВНЫЕ ОРГАНИЧЕСКИЕ, А...",вещества поверхностно активные органические а...
...,...,...,...
4177894,2710,"МИНЕРАЛЬНОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ Д...",минеральное моторное масло всесезонное для д...
4177895,3403,"СИНТЕТИЧЕСКОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ...",синтетическое моторное масло всесезонное для...
4177896,3403,"СИНТЕТИЧЕСКОЕ МОТОРНОЕ МАСЛО, ВСЕСЕЗОННОЕ, ДЛЯ...",синтетическое моторное масло всесезонное для...
4177897,3403,"МАСЛО ТРАНСМИССИОННОЕ НА СИНТЕТИЧЕСКОЙ ОСНОВЕ,...",масло трансмиссионное на синтетической основе ...


In [88]:
print(user_df['OPISANIE'][1])
print(user_df['TNVED'][1])

ТРАНСФОРМАТОРЫ, МОЩНОСТЬЮ БОЛЕЕ 1 КВА, НО НЕ БОЛЕЕ 16 КВА, НЕ ЛОМ ЭЛЕКТРООБОРУДОВАНИЯ:
8504


In [77]:
STOPWORDS = set(stopwords.words('russian'))
MIN_WORDS = 4
MAX_WORDS = 200

PATTERN_S = re.compile("\'s;")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 

def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers.
        text (str): input text
    return (str): modified initial text
    """
    text = str(text).lower()  # lowercase text
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(sentence, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True):
    """
    Lemmatize, tokenize, crop and remove stop words.
    """
    if lemmatize:
        stemmer = WordNetLemmatizer()
        tokens = [stemmer.lemmatize(w) for w in word_tokenize(sentence)]
    else:
        tokens = [w for w in word_tokenize(sentence)]
    token = [w for w in tokens if (len(w) > min_words and len(w) < max_words
                                                        and w not in stopwords)]
    return tokens    


def clean_sentences_user(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['OPISANIE'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df

def clean_sentences_fz(df):
    """
    Remove irrelavant characters (in new column clean_sentence).
    Lemmatize, tokenize words into list of words (in new column tok_lem_sentence).
    """
    print('Cleaning sentences...')
    df['clean_sentence'] = df['OPISANIE_SPR'].apply(clean_text)
    df['tok_lem_sentence'] = df['clean_sentence'].apply(
        lambda x: tokenizer(x, min_words=MIN_WORDS, max_words=MAX_WORDS, stopwords=STOPWORDS, lemmatize=True))
    return df
    

In [81]:
clean_fz_df = clean_sentences_fz(fz_df)
clean_fz_df.to_csv('clean_fz_df.csv', sep=';', encoding='mbcs')
clean_fz_df

Cleaning sentences...


In [103]:
partial = pd.DataFrame()
uniques = user_df['TNVED'].unique().tolist()[:10]
for number in uniques:
    df = user_df[user_df['TNVED'] == number]
    partial = partial.append(df)
    
partial_clean = clean_sentences_user(partial)
partial_clean.to_csv('partial_clean_df.csv', sep=';', encoding='mbcs')
partial_clean

In [104]:
def extract_best_indices(m, topk, mask=None):
    """
    Use sum of the cosine distance over all tokens.
    m (np.array): cos matrix of shape (nb_in_tokens, nb_dict_tokens)
    topk (int): number of indices to return (from high to lowest in order)
    """
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score 
    if mask is not None:
        assert mask.shape == m.shape
        mask = mask[index]
    else:
        mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:topk]  
    return best_index

### Preparing user - fz data

In [129]:
partial_clean['KOD_SHORT'] = partial_clean['TNVED']

transformer_df = pd.DataFrame()
transformer_df = transformer_df.append(fz_df[['KOD_SHORT', 'clean_sentence']])
transformer_df = transformer_df.append(partial_clean[['KOD_SHORT', 'clean_sentence']])

X = transformer_df['clean_sentence'].values
Y = transformer_df['KOD_SHORT'].values

### Splitted before vectorizing

In [164]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

In [165]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Adapt stop words
token_stop = tokenizer(' '.join(STOPWORDS), lemmatize=False)

# Fit TFIDF
vectorizer = TfidfVectorizer(stop_words=token_stop, tokenizer=tokenizer) 
tfidf_mat = vectorizer.fit_transform(X_train) # -> (num_sentences, num_vocabulary)
tfidf_mat.shape

(154463, 69126)

In [141]:
def get_recommendations_tfidf(sentence, tfidf_mat):
    
    """
    Return the database sentences in order of highest cosine similarity relatively to each 
    token of the target sentence. 
    """
    # Embed the query sentence
    tokens = [str(tok) for tok in tokenizer(sentence)]
    vec = vectorizer.transform(tokens)
    # Create list with similarity between query and dataset
    mat = cosine_similarity(vec, tfidf_mat)
    # Best cosine distance for each token independantly
    print(mat.shape)
    best_index = extract_best_indices(mat, topk=10)
    return best_index

### Vectorizing matrix ready, time to test how it works on semi random data

In [166]:
len(X_test)

1561

In [ ]:
index = 7
query_sentence = X_test[index]
print(X_test[index])
print(y_test[index])

In [170]:
query_sentence = input('Введите запрос \n')
best_index = get_recommendations_tfidf(query_sentence, tfidf_mat)
print(best_index)
display(transformer_df[['KOD_SHORT', 'clean_sentence']].iloc[best_index][:15])

Введите запрос 
Каркасы
(1, 154463)
[136360  82740 126150  25308    356   9823 146661  39923  21185  77730]


,KOD_SHORT,clean_sentence
46507,9019,гидромассажная ванночка для ног
2252515,8481,клапаны регулирующие для маслогидравлических с...
4078601,3402,синтетические моющие средства гост 25644 96 н...
253721,8504,катушки постоянной индуктивности не является...
356,2074,спинки шейки спинки вместе с шейкой гузки и...
9823,8467,аппараты углошлифовальные
2062500,9405,части ламп и осветительного оборудования пост...
2522602,8504,катушки индуктивности и дроссели прочие катуш...
3290363,9025,изделия медицинского назначения
1918798,8481,части клапанов versa для газотурбинной установки


In [171]:
transformer_df

,KOD_SHORT,clean_sentence
0,1012,лошади живые чистопородные племенные животные
1,1012,лошади ослы мулы и лошаки живые прочие
2,1012,прочие лошади живые прочие
3,1013,ослы живые
4,1019,прочие мулы и лошаки живые
...,...,...
4176249,9405,изделия для электрического осветительного обор...
4176250,9405,бытовое электрическое осветительное оборудован...
4176251,9405,электрическое осветительное оборудование пото...
4176252,9405,бытовые лампы электрические настольные наполь...
